**Table of contents**<a id='toc0_'></a>

- [Die Webseite der TH Nürnberg- Intranet Scrapen](#toc1_)
  - [Einleitung](#toc1_1_)

- [Scrapen der home Seite](#toc1_2_)
  - [Links Filtern](#toc1_3_)
  - [Dublikate entfernen und sortieren](#toc1_4_)
  - [Links abspeichern](#toc1_5_)
  - [Downloaden der Files](#toc1_6_)
  - [Weitere Iterationstufen](#toc1_7_)
  - [Iteration](#toc1_8_)
  - [Texte extrahieren](#toc1_9_)<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

  

# <a id='toc1_'></a>[Die Webseite der TH Nürnberg- Intranet Scrapen](#toc_)


## <a id='toc1_1_'></a>[Einleitung](#toc0_)


Die Webseite der TH-Nürnberg wirkt als Ausgangspunkt für die Wissensgrundlage des Chatbots.

Conventions:

- pandas Spaltennamen im Sigular
- Die meißten Links sind keine URLs, da sie lokal sind


In [ ]:
import sys
sys.path.append('..')
from bs4 import BeautifulSoup
from bs4.element import Comment
import pandas as pd
from db_init import db_get_df, db_save_df
import glob
import json
import requests
import sys
from tqdm import tqdm

from urllib3.exceptions import InsecureRequestWarning
import urllib3
# Suppress only the single InsecureRequestWarning from urllib3 needed
urllib3.disable_warnings(InsecureRequestWarning)

**Note:** Um diesen Notebook auszuführen, braucht man eine VPN Verbindung zur TH-Intranet.


## <a id='toc1_2_'></a>[Scrapen der home Seite](#toc0_)


Als Ausganspunkt für unsere Daten nutzen wir die Home Webseite des TH-Nürnberg-Intranet. (https://intern.ohmportal.de/)
Diese Website downloaden wir und suchen alle Links auf andere Webseiten und speichern diese Links in eine Liste.
Als nächsten Schritt rufen wir alle Links aus dieser Liste auf und sammeln wiederum alle Links von jeder dieser Webseiten.
In der daraus resultierenden Liste sortieren wir alle Links aus, die nicht auf die Webseite des TH-Intranet verweisen.
Dann laden wir alle Dokumente herunter und speichern sie in der Datenbank.


Eine Funktion, die eine URL als parameter nimmt und das HTML file zurückgibt, falls die Seite existiert.

Dafür nutzen wir die requests Bibliothek.


Da einige Seiten eine Umleitung auf die THN Webseite enthalten, überprüfen wir, ob die URL nach dem Umleiten noch zur ursprünglichen Domäne gehört.


**Hinweis** SSL Zertigikat prüfung wird ausgesetzt, da einige Seiten deshalb ein Fehler werfen.
Bsp URL: "https://intern.ohmportal.de/institutionen/fakultaeten/betriebswirtschaft/online-services/page.html"


In [ ]:
def download_html_from_url(url):
    
    html = ""

    try:   
        res = requests.get(url,verify=False)
    except Exception as error:
        print("FEHLER:" , error)
        return html
    
    if res.status_code == 200:
        if res.url.startswith(url):
            html = res.text
        else:
            print(f"Umleitung zu externer Seite verhindert. URL: {res.url}")
    else:
        print(f"Kein Inhalt heruntergeladen. Statuscode: {res.status_code}")
    
    return html


Eine Funktion, die ein HTML file nach Links durchsucht und alle gefundenen externen und internen Links zurückgibt.
Dafür nutzen wir die Bibliothek Beautifulsoup, mit dem lxml parser.


In [ ]:
def get_links_from_one_html(html):
    soup = BeautifulSoup(html,"lxml")
    links = [a["href"] for a in soup.find_all('a', href=True)]
    return links

Jetzt geben wir unsere initiale URL an und extrahieren alle Links aus dieser


In [ ]:
BASE_URL = "https://intern.ohmportal.de/"
html = download_html_from_url(BASE_URL)
links = get_links_from_one_html(html)
df = pd.DataFrame({"link": links})
print(*df["link"])
print(len(df["link"]))

## <a id='toc1_3_'></a>[Links filtern](#toc0_)


Zunächst können wir alle Links überprüfen, ob sie Parameter oder sections mit

- _?param1=hallo_
- _#section_

enthalten. Beide Attribute sind für den Download der Webseiten nicht notwendig und werden ausgefiltert. Dies spart uns HTML Duplikate.


In [ ]:
def clean_links(links):
    cleaned_links = [link.split('#')[0].split('?')[0] for link in links]
    return cleaned_links

Nun speichern wir das zwischen Ergebnis der gefilterten Links.


In [ ]:
df["link"]=clean_links(df["link"])
print(*df["link"])


Nun löschen wir alle Links, die kein Inhalt haben (leere Links).


In [ ]:
def remove_empty_links(links):
    cleaned_links = [link for link in links if link.strip()]
    return cleaned_links

In [ ]:
df = pd.DataFrame({"link": remove_empty_links(df["link"])})
print(*df["link"])
print(len(df["link"]))

Man sieht, dass wir 0 Links ausgefiltert haben, wenn man die Anzahl der Links vor und nach der Filterung vergleicht.


## <a id='toc1_4_'></a>[Externe Links finden- nur zur Visualisierung (optionale Funktion)](#toc0_)###


Jetzt können wir Mal nachschauen, auf welche externen Seiten die Startseite des THN-Intranet verweist.


In [ ]:
def find_extern_urls(urls):
    external_links = []
    for link in urls:
        if link.startswith("http"):
            external_links.append(link)

    return external_links

In [ ]:
external_links = find_extern_urls(df["link"])
print("Anzahl externer Links: ", len(external_links))
print(external_links)

Die Seite verweist also auf elearning, auf jobboerse und ein Forum.


### <a id='toc1_5_'></a>[Interne Links filtern](#toc0_)


Wir filtern nun noch alle Links heraus, die keine HTML-inhalte besitzen, wie z.B. pdf oder xml Dateien, die mail Links enthalten oder die nicht auf die THN Webseite referieren.


In [ ]:
def filter_intern_links(urls):
    filtered_links = []
    for url in urls:
        if url.startswith("http"):
            continue
        elif url.startswith("mailto:"):
            continue
        elif url.startswith("javascript:"):
            continue
        elif url.startswith("&#"): # is encoded mailto
            continue
        elif ".xml" in url:
            continue
        elif ".docx" in url:
            continue
        elif ".pdf" in url:
            continue
        elif url == "/":
            continue
        elif url == "&":
            continue
        else:
            filtered_links.append(url)
    return filtered_links

In [ ]:
intern_links = filter_intern_links(df["link"])
print("Anzahl interner Links: ", len(intern_links))
print(intern_links)

Nun haben wir 4 Links entfernt.


## <a id='toc1_4_'></a>[Dublikate entfernen und sortieren](#toc0_)

Jetzt können wir die duplikate entfernen und anschließend alphabetisch sortieren.


In [ ]:
def sort_and_remove_dublicates(df):
    if 'html' in df.columns:
        df = df.sort_values(by='html', ascending=False)
        df = df.drop_duplicates(subset='link', keep='first')
        df = df.reset_index(drop=True)
    else:
        df = df.sort_values(by='link', ascending=False)
        df = df.drop_duplicates(subset='link', keep='first')
        df = df.reset_index(drop=True)
    return df

In [ ]:
intern_links = filter_intern_links(df["link"])
df = pd.DataFrame({"link": intern_links})
df = sort_and_remove_dublicates(df)
print("Anzahl interner Links (ohne Dublikate): ", len(df["link"]))
print(*df["link"])

Anhand der Anzahl der Links vor und nach der Duplikaten Entfernung, sieht man das 14 doppelt vorkommende Links entfernet werden.


## <a id='toc1_5_'></a>[Links abspeichern](#toc0_) 


Für unsere weiteren Schritte werden wir immer nur interne Links verwenden, deshalb speichern wir an dieser Stelle mal die internen Links ab.


In [ ]:
db_save_df(df, "only_links")

## <a id='toc1_6_'></a>[Downloaden der files](#toc0_)


Jetzt können wir mit dem downloaden anfangen.


Diese Funktion lädt nun alle Html files zu den Links herunter und speichert sie im Dataframe neben den "link" in einer Spalte "html".


In [ ]:
def download_all_urls(links):
    htmls = []
    for link in tqdm(links):
        url = "https://intern.ohmportal.de/" + link
        html = download_html_from_url(url)
        htmls.append(html)
    return htmls

In [ ]:
df["html"]=download_all_urls(df["link"])

In [ ]:
def remove_rows_with_empty_html(df):
    df = df[df["html"] != ""]  
    df.reset_index(drop=True, inplace=True)
    return df


In [ ]:
df = remove_rows_with_empty_html(df)

In [ ]:
print(len(df))

Nun sind 12 Reihen entfernt, die keinen HTML Inhalt hatten, da sie zum Beispiel auf eine andere Domäne verweisen wurden.


Wir können die Daten an dieser Stelle abspeichern.


In [ ]:
db_save_df(df, "intranet_html_iter_01")

## <a id='toc1_7_'></a>[Weitere Iterationsstufen](#toc0_) 


Wenn wir ab diesem Abschnitt starten können wir die vorher gesammelten Daten neu laden.


In [ ]:
df = db_get_df("intranet_html_iter_01")
print(len(df["link"]))

In [ ]:
non_none_html_rows = df[df["html"].notnull()]  # Filtere die Zeilen, in denen "html" nicht "None" ist
print(len(non_none_html_rows))

Jetzt können wir die heruntergeladenen HTML files nach weiteren Links durchsuchen und Sie dem Dataframe hinzufügen


In [ ]:
def find_all_links_in_html(htmls):
    all_links = []
    for html in tqdm(htmls):
        links = get_links_from_one_html(html)
        links= clean_links(links)
        links= remove_empty_links(links)
        links= filter_intern_links(links)
        [all_links.append(link) for link in links]
    return all_links

In [ ]:
all_links = find_all_links_in_html(df["html"])
len(all_links)

Wir haben jetzt also 979 Links gesammelt, davon sind aber viele Dublikate.


Nun führen wir die neu gesammelten Links mit den ursprünglichen Links zusammen, wobei die neuen Links ein "None" Wert für die "html" Spalte bekommen.


In [ ]:
df_new = pd.DataFrame({"link": all_links, "html": None})
df = pd.concat([df, df_new])

In [ ]:
df = sort_and_remove_dublicates(df)
len(df["link"])

Gefiltert nach dublikaten haben wir nun also noch 285 Links


In [ ]:
print(*df["link"])

In [ ]:
def print_unique_link_endings(df):
    endings = set() 
    
    for link in df["link"]:
        parts = link.split("/") 
        if len(parts) > 0:
            ending = parts[-1]  
            endings.add(ending)
    
    for ending in endings:
        print(ending)

In [ ]:
print_unique_link_endings(df)

In [ ]:
non_none_html_rows = df[df["html"].notnull()]  # Filtere die Zeilen, in denen "html" nicht "None" ist
print(len(non_none_html_rows))

Um den Daten nun weitere Webseiten hinzuzufügen, können wir für jede weitere URL schauen, ob sie schon heruntergeladen wurde. Wenn nicht, dann laden wir sie jetzt herunter.


In [ ]:
def update_df_with_html(df):
    for index, row in tqdm(df.iterrows()):
        if pd.isna(row['html']) or row['html'] == '':
            url = "https://intern.ohmportal.de/" + row["link"]
            html = download_html_from_url(url)
            df.at[index, 'html'] = html
    
    return df

In [ ]:
df = update_df_with_html(df)

Nun entfernen wir die leeren HTML Inhalte.


In [ ]:
df=remove_rows_with_empty_html(df)

In [ ]:
non_none_html_rows = df[  df["html"]=="" ]  # Filtere die Zeilen, in denen "html" nicht "None" ist
print(len(non_none_html_rows))

Nun speichern wir die 2. Iteration.


In [ ]:
db_save_df(df, "intranet_html_iter_02")

In [ ]:
df = db_get_df("intranet_html_iter_02")
print(len(df["link"]))

In [ ]:
def remove_broken_link(df):
    df = df[df['link'] != '/seitenbaum/studierende/einrichtungen-beratung/language-center/page.html']
    df = df.reset_index(drop=True)
    return df

## <a id='toc1_8_'></a>[Iterationen](#toc0_)


1. Iteration: Startseite TH-Intranet bzw. neue Links
2. Iteration: Links von startseite aufrufen und HTML scrapen
3. Iteration: Links von diesen Seiten aufrufen und HTML scrapen


In [ ]:
def iteration(df):
    all_links = find_all_links_in_html(df["html"])
    df_new = pd.DataFrame({"link": all_links, "html": None})
    df = pd.concat([df, df_new])
    df = sort_and_remove_dublicates(df)
    print(f"got {len(df[df['html'].isna()])} new links")
    df=remove_broken_link(df)
    df = update_df_with_html(df)
    df=remove_rows_with_empty_html(df)
    return df

In [ ]:
df = db_get_df("intranet_html_iter_03")

In [ ]:
df=iteration(df)

In [ ]:
print(len(df))

In [ ]:
all_links = find_all_links_in_html(df["html"])
df_new = pd.DataFrame({"link": all_links, "html": None})
df = pd.concat([df, df_new])
df = sort_and_remove_dublicates(df)
print(f"got {len(df[df['html'].isna()])} new links")

In [ ]:
df=remove_broken_link(df)

In [ ]:
df = update_df_with_html(df)
df=remove_rows_with_empty_html(df)

In [ ]:
db_save_df(df, "intranet_html_iter_04")

## <a id='toc1_9_'></a>[Texte extrahieren](#toc0_)


Als nächstes müssen wir aus den rohen HTML Dokumenten die unrelevanten Daten aussortieren, wie holen die letzten verfügbaren Iterations Daten.


In [ ]:
df = db_get_df("intranet_html_iter_04")

Die nachfolgende Funktion bestimmt, ob ein Beautifulsoup geparstes HTML Element sichtbar ist oder nicht.


In [ ]:
def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True

Jetzt bestimmen wir eine Funktion, die ein HTML als Input bekommt und daraus die Texte und Titeln bestimmt.

Die HTML Seiten haben folgenden Strukturen:

- "main" (https://intern.ohmportal.de/seitenbaum/home/page.html)

main>div#main-coloumn>dicv>#idcontent-coloumn

In [142]:
from bs4 import BeautifulSoup, Comment

def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True

def get_content(file):
    soup = BeautifulSoup(file, "lxml")

    title = soup.find("title")
    if title:
        title = title.text
    else:
        title = ""

    main = soup.find("div", {'id': 'main'})
    
    visible_texts = ""
    if main:
        container = main.find("div", {'id': 'contentColumn'})
        if container:
            texts = container.find_all(text=True)
            visible_texts = filter(tag_visible, texts)
            visible_texts = [t.strip() for t in visible_texts if t.strip()]  # Entferne Leerzeichen und leere Zeichenfolgen
            visible_texts = ' '.join(visible_texts)

    return {
        "title": title,
        "text": visible_texts
    }


Nun speichern wir die Texte sowie die dazugehörigen Titeln in den Dataframe.


In [143]:
parsed_texts = []
titles = []
for html in tqdm(df["html"]):
    content = get_content(html)
    parsed_texts.append(content["text"])
    titles.append(content["title"])

df["text"] = parsed_texts
df["title"] = titles

  0%|          | 2/1686 [00:00<01:37, 17.24it/s]C:\Users\lizab\AppData\Local\Temp\ipykernel_10120\1933498786.py:25: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  texts = container.find_all(text=True)
100%|██████████| 1686/1686 [00:10<00:00, 159.05it/s]


In [145]:

print(df["text"])

0                                                        
1                                                        
2       Donnerstag, 04. Juli 2019 finanziert neues eBo...
3       Öffnungszeiten Ab hier finden Sie die Öffnungs...
4       Ältestenrat Aufgaben Der Ältestenrat unterstüt...
                              ...                        
1681    Ausgabe der Abschlussarbeit Auf dieser und den...
1682    Anmeldung der Abschlussarbeit 1. Zur Erfassung...
1683    Abgabe der Abschlussarbeit 1. rechtzeitige Abg...
1684    Wiederholung der Abschlussarbeit Wird eine Bac...
1685    Bewertung der Abschlussarbeit Zur Bewertung de...
Name: text, Length: 1686, dtype: object


In [146]:
print(df["title"])

0                                                        
1                                                        
2                          Technische Hochschule Nürnberg
3       Technische Hochschule Nürnberg: Öffnungszeiten...
4             Technische Hochschule Nürnberg: Ältestenrat
                              ...                        
1681    Technische Hochschule Nürnberg: Ausgabe der Ab...
1682    Technische Hochschule Nürnberg: Anmeldung der ...
1683    Technische Hochschule Nürnberg: Abgabe der Abs...
1684    Technische Hochschule Nürnberg: Wiederholung d...
1685    Technische Hochschule Nürnberg: Bewertung der ...
Name: title, Length: 1686, dtype: object


In [147]:
db_save_df(df, "intranet_html_attributes")